# US presidential statistics
## Simon Repko, Lukas Petrasek
### IES FSS CU
### 31.5.2019

This notebook serves as a demonstration of a school project whose goal is to achieve the following:
* scrape web pages to get historical data on US presidents
* manipulate the data into a form suitable for being visualized
* make vizualizations based the data

In [ ]:
# TODO: import packages

In [ ]:
# TODO: initialize the scraping class and apply the methods necessary to get the data here

In [ ]:
# TODO: manipulate the data here

In [ ]:
# TODO: make the visualizations here

In [ ]:
# TODO: conclude here?